In [44]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

In [45]:
def basePrep(df):
	df['rndf'] = df['rndf'].map({"Yes": 1, "No": 0})
	df['pdynamic'] = df['pdynamic'].map({'new_entrant': 1, 'incumbent': 0})	
	df["SNATION_PARTAL".lower()] = df["SNATION_PARTAL".lower()].map({"Y": 1, "N": 0})
	df['cr_bor_part'] = df['cr_bor_part'].map({"Y": 1, "N": 0})
	df['mfgf'] = df['mfgf'].map({"Yes": 1, "No": 0})
	df['jvf'] = df['jvf'].map({'Yes': 1, 'No': 0})
	
	cols = ['rndf', 'jvf','ddist_bin','ddist_int', 'ddist_abs', 
		'avg_emp_pp', 'public_count', 'pdynamic', "SNATION_PARTAL".lower(), 
		'cr_bor_part', 'mfgf', 'avg_emp_pp_log']
	df = df[cols].dropna()
	
	return df

def variation(df):
	cols = ['rndf', 'ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count', 
			"SNATION_PARTAL".lower(), 'cr_bor_part', 'mfgf']
	X = (df[cols]).astype(float)
	y = df['pdynamic'].astype(float)
	X_constant = sm.add_constant(X, prepend=False)

	logit_model = GLM(y, X_constant, family=families.Binomial())
	logit_results = logit_model.fit()
	print(logit_results.summary())

## Try all potential variables

In [46]:
df = pd.read_csv('./data/JV_data_dist.csv')
df = basePrep(df)

variation(df)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  729
Model:                            GLM   Df Residuals:                      719
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -433.35
Date:                Thu, 16 Jun 2022   Deviance:                       866.71
Time:                        17:20:04   Pearson chi2:                     739.
No. Iterations:                    19   Pseudo R-squ. (CS):             0.1188
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               0.2668      0.310      0.

## Filter only R&D alliances

In [47]:
df2 = pd.read_csv('./data/JV_data_dist.csv')

df2 = df2[df2['rndf'] == "Yes"]
df2 = basePrep(df2)

variation(df2)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                   66
Model:                            GLM   Df Residuals:                       58
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -33.639
Date:                Thu, 16 Jun 2022   Deviance:                       67.277
Time:                        17:20:05   Pearson chi2:                     64.3
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1657
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               2.2089      0.864      2.

## Filter for alliance SIC

In [48]:
df3 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df3 = df3[df3['sic'].str.contains('3711') == True]
df3 = basePrep(df3)

variation(df3)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  311
Model:                            GLM   Df Residuals:                      302
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -188.09
Date:                Thu, 16 Jun 2022   Deviance:                       376.17
Time:                        17:20:06   Pearson chi2:                     325.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1536
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.5452      0.694     -0.

## Filter for same nation participants and alliance

In [49]:
df4 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df4 = df4[df4['snation_partal'] == "Y"]
df4 = basePrep(df4)

variation(df4)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  154
Model:                            GLM   Df Residuals:                      146
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -74.811
Date:                Thu, 16 Jun 2022   Deviance:                       149.62
Time:                        17:20:06   Pearson chi2:                     155.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.06536
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.5447      0.661     -0.

## Filter for Manufacturing agreement flags

In [50]:
df5 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df5 = df5[df5['mfgf'] == "Yes"]
df5 = basePrep(df5)

variation(df5)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  481
Model:                            GLM   Df Residuals:                      473
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -295.78
Date:                Thu, 16 Jun 2022   Deviance:                       591.56
Time:                        17:20:07   Pearson chi2:                     492.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1188
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               0.6032      0.483      1.

In [51]:
df6 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df6 = df6[df6['mfgf'] == "Yes"]
df6 = df6[df6['sic'].str.contains('3711') == True]
df6 = basePrep(df6)

variation(df6)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  267
Model:                            GLM   Df Residuals:                      259
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -162.44
Date:                Thu, 16 Jun 2022   Deviance:                       324.88
Time:                        17:20:07   Pearson chi2:                     278.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1472
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.2747      0.830     -0.

In [52]:
df7 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df7 = df7[df7['mfgf'] == "Yes"]
df7 = df7[df7['sic'].str.contains('3711') == True]
df7 = df7[df7['public_count'] >= 1]
df7 = df7[df7['nump'] == 2]
df7 = basePrep(df7)

variation(df7)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  202
Model:                            GLM   Df Residuals:                      194
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -119.70
Date:                Thu, 16 Jun 2022   Deviance:                       239.41
Time:                        17:20:07   Pearson chi2:                     205.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1663
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.6154      1.292     -0.

## Filter for single country

In [68]:
df8 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df8 = df8[df8['natc'] == "IN"]

df8 = basePrep(df8)

variation(df8)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                   76
Model:                            GLM   Df Residuals:                       67
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -31.956
Date:                Thu, 16 Jun 2022   Deviance:                       63.913
Time:                        17:35:48   Pearson chi2:                 1.23e+03
No. Iterations:                     6   Pseudo R-squ. (CS):             0.2867
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -1.8852      1.588     -1.

## Test for final H1

In [53]:
dfH = pd.read_csv("./data/JV_data_dist.csv")

dfH = dfH[dfH['mfgf'] == "Yes"]
dfH = dfH[dfH['sic'].str.contains('3711') == True]
dfH = dfH[dfH['public_count'] >= 1]
dfH = dfH[dfH['nump'] == 2]
dfH = dfH[dfH['avg_emp_pp'] >= 8]
dfH = basePrep(dfH)

cols = ['ddist_int']
X = (dfH[cols]).astype(float)
y = dfH['pdynamic'].astype(float)
X_constant = sm.add_constant(X, prepend=False)

logit_model = GLM(y, X_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  202
Model:                            GLM   Df Residuals:                      200
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -130.99
Date:                Thu, 16 Jun 2022   Deviance:                       261.97
Time:                        17:20:08   Pearson chi2:                     202.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.06772
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ddist_int      0.0004      0.000      3.632      0.0

## Test for final H2

In [58]:
dfH2 = pd.read_csv("./data/JV_data_dist.csv")

dfH2 = dfH2[dfH2['mfgf'] == "Yes"]
dfH2 = dfH2[dfH2['sic'].str.contains('3711') == True]
dfH2 = dfH2[dfH2['public_count'] >= 1]
dfH2 = dfH2[dfH2['nump'] == 2]
dfH2 = dfH2[dfH2['avg_emp_pp'] >= 30000]

dfH2 = basePrep(dfH2)

cols = ['ddist_int', 'public_count', 'rndf', 'snation_partal', 'avg_emp_pp', 'avg_emp_pp_log']
X = (dfH2[cols]).astype(float)
y = dfH2['jvf'].astype(float)
X_constant = sm.add_constant(X, prepend=False)

logit_model = GLM(y, X_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    jvf   No. Observations:                  116
Model:                            GLM   Df Residuals:                      109
Model Family:                Binomial   Df Model:                            6
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -61.895
Date:                Thu, 16 Jun 2022   Deviance:                       123.79
Time:                        17:21:38   Pearson chi2:                     114.
No. Iterations:                    20   Pseudo R-squ. (CS):             0.1330
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
ddist_int          0.0004      0.000      2.